## Different Types of Debugging

- **Coding Mistakes**: when your model does not do what you want it to do (python, pytorch, math or logic errors)
- **Training Mistakes**: when your model does what you want it to do, but is not learning well.
- **Testing/Decoding Mistakes**: when your model is learning well but outputs bad results.

## Coding Errors

**Symptoms**

- Loss does not decrease at all - obviously you need to plot the loss against epoches to show this
- Output are constants
- Training stops mid-time for unclear reasons.
- Time issue: when everything works but is too slow. In your epochs, use the time module to check the duration of all your subtasks (data loading, forward, backward,...), and find the aberrant one.

**Typical Mistakes**

- You forgot to put your model in eval mode during inference time and your model is now producing garbage.
- You passed softmax outputs to a loss that expects raw logits. E.g. passing softmax outputs to `nn.CrossEntropyLoss()`.

**Advice**

- Print everything to look for the first moment the problem appears. Be methodical.
- Check your data: Not iterating ? Instance-label misalignment ? (spend a good amount of time checking if your data is sane. This is crucial especially if you’re doing some preprocessing on your data before passing it to your network.)
- Check your shapes: everything consistent? (What does this mean?)
- Check your hyperparameters: when you print them, are they what they are supposed to be?
- Remember to turn off regularization - don't let them overwhelm your data.

## Training Errors

**Symptoms**

For those errors, usually your loss does decrease, but not enough. Or when the training errors decrease but testing/validation errors do not. Or your model learns, training and validation loss decrease, but accuracy is low.

 > If you see absurdly low performance (or even random), it is probably a coding error. A good rule of thumb to check whether it is a training or coding error is that a random classification model would have a cross-entropy loss of approximately $\log(\text{number of classes})$.

**Typical Mistakes**

- Modelization issues : your model is too small to have sufficient capacity to learn patterns (or not well designed when the problem is complex)
- Optimization issues : you cannot train your model properly
- Overfitting : your model is too big/you train too long

**Advice**

- Try a few datapoints or a random subset of the training data, and try to see if your model can overfit it, i.e. training accuracy goes to 100% and validation accuracy much less. This can assure and inform if your model has enough capacity, and also beneficial in debugging coding errors too. Note that you need to first disable regularizations and other limiting features. Once you are certain your training loss decreases to 0, you can slowly add back regularizations and increase the data size (for which you should see loss increasing) and start fine tuning.
- Check the below for optimization issues:
 > Learning rate: if too small, you will learn too slowly. If too large, you will learn for a while then diverge. Default “good” : 0.001. It is recommended to do learning rate decay : start large, then decrease (for
example when loss stops improving).

 > Optimizer: (default “good” : Adam)
 
 > Initialization: (default “good” : xavier)

 > Batching (just the batch size on simple problems). Default “good” : from 32 to 256 if you can afford it.

 > Too deep models can create optimization problems too (vanishing gradients). They can also lead to overfitting.
 
 > Look at the activation/grad distribution per layer - recall the motivation of BatchNorm: it is not good to see any strange distribution.

- Use the following tricks when you have an overfitting problem:

 > Verify that you shuffle your training data
 
 > Decrease your model size/depth
 
 > Use some of the tricks you know that help generalization : dropout, batchnorm, early stopping, validation-driven learning rate decay. Note : adaptative optimizers (Adam,...) overfit more.
 
 > It is also possible to overfit on the validation set. This happens when you try a very large amount of architectures/hyperparameters with the same validation set : you may find one that works “by chance” and won’t generalize. If you plan to look for many architectures, consider a better validation method like K-fold.
 
- In classification problem, keep an eye on the accuracy, since the loss function is usually an approximation to the (step and indifferentiable) accuracy function.

## TensorBoard for Visualization

TensorBoard is a neural network visualization library developed by Google as part of Tensorflow. In the past, people can use TensorBoard in PyTorch via third-party adaptors like tensorboardX. Starting from 1.2.0 (the latest version), **PyTorch officially supports TensorBoard**. We recommend you to use the latest version of PyTorch and use its built-in support of TensorBoard for visualization.

This tutorial covers how to use PyTorch's official support of TensorBoard. You can also refer to the [official documentation](https://pytorch.org/docs/stable/tensorboard.html). If you insist on using an older version of PyTorch, try [tensorboardX](https://github.com/lanpa/tensorboardX).

Let's take up the same task as defined in Recitation 2. We'll be training a Neural Network to classify if a set of points $(x_1, x_2)$ lie inside a circle of radius $1$ or not. For more details on what the task is, please re-visit Recitation 2.


In [0]:
# Install required libraries
!pip install torch>=1.2.0 tensorboard future tqdm

In [0]:
import torch
import torch.nn as nn

Similar to Recitation 2, we first sample some polar co-ordinates that are randomly distributed within a circle of radius 2 and centered at origin, ie. $(0,0)$.

In [0]:
import math

def sample_points(n):
    """
    :param n: Total number of data-points
    :return: A tuple (X,y) where X is a float tensor with shape (n,2)
               and y is an interger tensor with shape(n,)
    """    
    radius = torch.rand(n) * 2
    angle = torch.rand(n) * 2 * math.pi
    x1 = radius * angle.cos()
    x2 = radius * angle.sin()
    y = radius < 1
    x = torch.stack([x1, x2], dim=1)
    return x, y

In [0]:
# Generating the data

X_train, y_train = sample_points(10000)
X_val,y_val = sample_points(500)

print(X_train.size(), y_train.size())

torch.Size([10000, 2]) torch.Size([10000])


In [0]:
# Build a simple MLP
def build_model(dims, activation):
  layers = []
  for i in range(len(dims) - 1):
    layers.append(nn.Linear(dims[i], dims[i + 1]))
    if i < len(dims) - 2:
      layers.append(activation())
  return nn.Sequential(*layers) # Recall that if you don't do this, pytorch will not automatically register all params in the module.

# Test the function
print(build_model([2, 12, 1], nn.Sigmoid))

Sequential(
  (0): Linear(in_features=2, out_features=12, bias=True)
  (1): Sigmoid()
  (2): Linear(in_features=12, out_features=1, bias=True)
)


A SummaryWriter writes all values we want to visualize to a given directory. This line creates a SummaryWriter that creates write event files and saves in the `./runs/example` directory.
```python
from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter("./runs/example")
```
**You should use different run directories in a common root directory for different runs of your model.** TensorBoard looks for runs in the root directory. So for this example, we start the TensorBoard with:

```sh
tensorboard --logdir=./runs
```
Then, visit `localhost:6006` with your browser to see the TensorBoard.

Each time we add a value, we specify a **tag** and a **step**. Each tag is a string and corresponds to a plot on TensorBoard. The step is an integer (`epoch` in this example) that serves as the X axis on the plot.

To plot a single scalar, use [*SummaryWriter.add_scalar()*](https://pytorch.org/docs/stable/tensorboard.html#torch.utils.tensorboard.writer.SummaryWriter.add_scalar). To plot multiple scalars on a plot, use [*SummaryWriter.add_scalars()*](https://pytorch.org/docs/stable/tensorboard.html#torch.utils.tensorboard.writer.SummaryWriter.add_scalars) and pass in a dict of scalars.

Using [*SummaryWriter.add_histogram()*](https://pytorch.org/docs/stable/tensorboard.html#torch.utils.tensorboard.writer.SummaryWriter.add_histogram) to plot a histogram of values in a tensor is also useful for understanding the dynamics of the network.

In [0]:
from tqdm import tqdm

def train(model, writer, epochs=1000):
  criterion = nn.BCEWithLogitsLoss()
  optimizer = torch.optim.Adam(model.parameters())
  for epoch in tqdm(range(epochs)):
    model.zero_grad()
    out = model(X_train).flatten()
    loss = criterion(out, y_train.float())
    train_loss = loss.item()
    train_acc = ((out > 0) == y_train).float().mean().item()
    
    loss.backward()
    # Plot histogram of gradient of all parameters
    for name, param in model.named_parameters():
      writer.add_histogram('grad_' + name, param.grad.data, epoch)
    optimizer.step()
    
    with torch.no_grad():
      out = model(X_val).flatten()
      val_loss = criterion(out, y_val.float()).item()
      val_acc = ((out > 0) == y_val).float().mean().item()
    # Plot loss and accuracy on train and val
    writer.add_scalars('loss', {'train': train_loss, 'val': val_loss}, epoch)
    writer.add_scalars('acc', {'train': train_acc, 'val': val_acc}, epoch)

## Using Different Activation functions

Let's see and understand how each of these activation functions perform.

- Sigmoid
    * Get values between 0 and 1.
    * A Sigmoid layer easily dies or saturates. A value too small kills the gradient flow whereas a value too big saturates the neurons, effectively passing no information through it.
    
- Tanh
    * Outputs values between -1 and 1. Also zero centered and so does not have the problem of all positive/negative gradients.
    * Better than Sigmoid but problem of saturation persists.

- ReLU
    * Converges quickly as is a threshold based activation and does not saturate.
    * Neurons die off. Large weight update could set the weights in such a way (they become negative) during backpropagation that they never fire for any data point. Important to set lower learning rates for ReLU.
    
    
**TRY IT OUT**

Use all the 3 activation functions. See which performs better and try to find out why. 

In [0]:
from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter("./runs/sigmoid")
model = build_model([2, 12, 1], nn.Sigmoid)
train(model, writer)

writer = SummaryWriter("./runs/tanh")
model = build_model([2, 12, 1], nn.Tanh)
train(model, writer)

writer = SummaryWriter("./runs/relu")
model = build_model([2, 12, 1], nn.ReLU)
train(model, writer)

100%|██████████| 1000/1000 [00:13<00:00, 72.27it/s]


Open TensorBoard and see the result! (These graphs can be found in Recitation 5 of )

![TensorBoard: accuracy](tensorboard_acc.png)

![TensorBoard: gradient distribution](tensorboard_grad.png)